### 1: Install Required Libraries

In [1]:
# Install required libraries (run this first)
%pip install optuna xgboost lightgbm catboost scikit-learn imbalanced-learn shap plotly matplotlib seaborn

   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
    ----------------

  You can safely remove it manually.
  You can safely remove it manually.


In [2]:

# Import all necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Machine Learning
from sklearn.model_selection import KFold, StratifiedKFold, cross_validate
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix, matthews_corrcoef, classification_report
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.base import clone

# Imbalanced Learning
from imblearn.ensemble import BalancedRandomForestClassifier, EasyEnsembleClassifier, BalancedBaggingClassifier

# Gradient Boosting
import xgboost as xgb
import lightgbm as lgb
import catboost as cb

# Hyperparameter Optimization
import optuna
from optuna.samplers import TPESampler

# Feature Selection
from sklearn.feature_selection import SelectKBest, mutual_info_classif, RFECV

# Visualization
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# Interpretation
import shap

print("All libraries imported successfully!")

All libraries imported successfully!


### Step 2: Enhanced Data Preprocessing with Advanced Techniques

In [3]:
# Load Training Dataset
data = pd.read_csv('DIA_trainingset_RDKit_descriptors.csv')

# extract features and target variable
X_train = data.iloc[:, 2:]
Y_train = data.iloc[:, 0]

# Load Test Dataset
test_data = pd.read_csv('DIA_testset_RDKit_descriptors.csv')
X_test = test_data.iloc[:, 2:]
Y_test = test_data.iloc[:, 0]

In [5]:
X_train.head()

,BalabanJ,BertzCT,Chi0,Chi0n,Chi0v,Chi1,Chi1n,Chi1v,Chi2n,Chi2v,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,1.821,1266.407,22.121,16.781,16.781,14.901,9.203,9.203,6.668,6.668,...,0,0,0,0,0,0,0,0,0,0
1,2.363,490.434,11.707,8.752,9.569,7.592,4.854,5.670,3.545,4.661,...,0,0,0,0,0,0,0,1,0,1
2,3.551,93.092,6.784,5.471,5.471,3.417,2.420,2.420,2.820,2.820,...,0,0,0,0,0,0,0,0,0,0
3,2.076,1053.003,21.836,16.995,16.995,14.274,9.926,9.926,7.662,7.662,...,0,0,0,0,0,0,0,0,0,0
4,2.888,549.823,14.629,9.746,9.746,8.752,5.040,5.040,3.601,3.601,...,0,0,0,0,0,0,0,0,0,0


In [7]:
X_test.head()

,BalabanJ,BertzCT,Chi0,Chi0n,Chi0v,Chi1,Chi1n,Chi1v,Chi2n,Chi2v,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,1.484,743.207,21.466,18.764,18.764,14.292,12.106,12.106,10.736,10.736,...,0,0,0,0,0,0,0,0,0,0
1,1.472,868.947,21.140,16.736,17.553,14.453,10.268,11.084,7.662,8.746,...,0,0,0,0,0,0,0,0,0,0
2,0.837,1409.004,39.189,32.904,32.904,26.011,20.941,20.941,18.816,18.816,...,0,0,0,0,0,0,0,0,0,0
3,2.406,621.298,13.828,10.297,10.297,9.092,5.847,5.847,4.217,4.217,...,0,0,0,0,0,0,0,0,0,0
4,1.320,2127.996,37.955,30.849,31.666,25.910,18.066,19.115,14.930,16.060,...,1,0,0,0,0,0,0,0,0,0


In [10]:
def advanced_preprocessing(X_train, X_test, y_train, method='robust'):
    """
    Advanced preprocessing with multiple scaling options and outlier handling
    
    Parameters:
    - method: 'standard', 'robust', 'minmax', 'quantile'
    """
    from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler, QuantileTransformer
    
    scalers = {
        'standard': StandardScaler(),
        'robust': RobustScaler(),
        'minmax': MinMaxScaler(),
        'quantile': QuantileTransformer(output_distribution='normal')
    }
    
    scaler = scalers[method]
    
    # Apply scaling
    X_train_scaled = pd.DataFrame(
        scaler.fit_transform(X_train),
        columns=X_train.columns,
        index=X_train.index
    )
    
    X_test_scaled = pd.DataFrame(
        scaler.transform(X_test),
        columns=X_test.columns,
        index=X_test.index
    )
    
    print(f"Applied {method} scaling")
    return X_train_scaled, X_test_scaled, scaler

def remove_highly_correlated_features(X, threshold=0.95):
    """
    Remove highly correlated features using advanced correlation analysis
    """
    corr_matrix = X.corr().abs()
    upper_triangle = corr_matrix.where(
        np.triu(np.ones(corr_matrix.shape), k=1).astype(bool)
    )
    
    # Find features to drop
    to_drop = [column for column in upper_triangle.columns 
               if any(upper_triangle[column] > threshold)]
    
    X_reduced = X.drop(columns=to_drop)
    print(f"Removed {len(to_drop)} highly correlated features (threshold: {threshold})")
    print(f"Features reduced from {X.shape[1]} to {X_reduced.shape[1]}")
    
    return X_reduced, to_drop

# Apply advanced preprocessing
X_train_enhanced, X_test_enhanced, scaler = advanced_preprocessing(
    X_train, X_test, Y_train, method='robust'
)

# Remove highly correlated features
X_train_final, dropped_features = remove_highly_correlated_features(
    X_train_enhanced, threshold=0.95
)
X_test_final = X_test_enhanced.drop(columns=dropped_features)

print(f"Final dataset shape: Training {X_train_final.shape}, Test {X_test_final.shape}")

Applied robust scaling
Removed 31 highly correlated features (threshold: 0.95)
Features reduced from 196 to 165
Final dataset shape: Training (477, 165), Test (120, 165)


### Step 3: Advanced Feature Selection with Multiple Methods

In [11]:
class AdvancedFeatureSelector:
    """
    Comprehensive feature selection using multiple methods
    """
    
    def __init__(self, random_state=42):
        self.random_state = random_state
        self.selected_features = {}
        
    def mutual_information_selection(self, X, y, k='auto'):
        """Enhanced mutual information selection"""
        if k == 'auto':
            k = min(50, X.shape[1] // 2)  # Adaptive k selection
            
        mi_scores = mutual_info_classif(X, y, random_state=self.random_state)
        feature_scores = pd.Series(mi_scores, index=X.columns).sort_values(ascending=False)
        
        # Select top k features
        selected_features = feature_scores.head(k).index.tolist()
        self.selected_features['mutual_info'] = selected_features
        
        return selected_features, feature_scores
    
    def recursive_feature_elimination(self, X, y, estimator=None):
        """RFECV with cross-validation"""
        if estimator is None:
            estimator = RandomForestClassifier(n_estimators=100, random_state=self.random_state)
            
        rfecv = RFECV(
            estimator=estimator,
            step=1,
            cv=5,
            scoring='roc_auc',
            n_jobs=-1
        )
        
        rfecv.fit(X, y)
        selected_features = X.columns[rfecv.support_].tolist()
        self.selected_features['rfecv'] = selected_features
        
        return selected_features, rfecv
    
    def variance_threshold_selection(self, X, threshold=0.01):
        """Remove low variance features"""
        from sklearn.feature_selection import VarianceThreshold
        
        selector = VarianceThreshold(threshold=threshold)
        selector.fit(X)
        selected_features = X.columns[selector.get_support()].tolist()
        self.selected_features['variance'] = selected_features
        
        return selected_features, selector
    
    def statistical_selection(self, X, y, method='f_classif', k=50):
        """Statistical feature selection"""
        from sklearn.feature_selection import f_classif, chi2
        
        if method == 'f_classif':
            selector = SelectKBest(f_classif, k=k)
        elif method == 'chi2':
            # Ensure non-negative values for chi2
            X_positive = X - X.min() + 1e-5
            selector = SelectKBest(chi2, k=k)
            X = X_positive
            
        selector.fit(X, y)
        selected_features = X.columns[selector.get_support()].tolist()
        self.selected_features['statistical'] = selected_features
        
        return selected_features, selector
    
    def ensemble_selection(self, X, y, methods=['mutual_info', 'rfecv', 'statistical']):
        """Combine multiple selection methods"""
        all_selected = []
        
        if 'mutual_info' in methods:
            features, _ = self.mutual_information_selection(X, y)
            all_selected.extend(features)
            
        if 'rfecv' in methods:
            features, _ = self.recursive_feature_elimination(X, y)
            all_selected.extend(features)
            
        if 'statistical' in methods:
            features, _ = self.statistical_selection(X, y)
            all_selected.extend(features)
            
        # Count feature frequency
        feature_counts = pd.Series(all_selected).value_counts()
        
        # Select features that appear in at least 2 methods
        ensemble_features = feature_counts[feature_counts >= 2].index.tolist()
        self.selected_features['ensemble'] = ensemble_features
        
        return ensemble_features, feature_counts

# Apply advanced feature selection
feature_selector = AdvancedFeatureSelector(random_state=42)

# Run ensemble selection
ensemble_features, feature_counts = feature_selector.ensemble_selection(
    X_train_final, Y_train, methods=['mutual_info', 'rfecv', 'statistical']
)

print(f"Ensemble selection chose {len(ensemble_features)} features")
print(f"Top 10 most selected features:")
print(feature_counts.head(10))

# Create final feature set
X_train_selected = X_train_final[ensemble_features]
X_test_selected = X_test_final[ensemble_features]

print(f"Final feature set shape: {X_train_selected.shape}")

Ensemble selection chose 75 features
Top 10 most selected features:
EState_VSA9          3
EState_VSA1          3
NumAliphaticRings    3
PEOE_VSA6            3
EState_VSA4          3
PEOE_VSA9            3
SlogP_VSA10          3
fr_urea              3
EState_VSA10         3
fr_piperdine         3
Name: count, dtype: int64
Final feature set shape: (477, 75)


### Step 4: Advanced Model Development with Optuna Optimization

In [ ]:
class OptunaModelOptimizer:
    """
    Advanced hyperparameter optimization using Optuna
    """
    
    def __init__(self, n_trials=100, cv_folds=5, random_state=42):
        self.n_trials = n_trials
        self.cv_folds = cv_folds
        self.random_state = random_state
        self.best_params = {}
        self.study_results = {}
    
    def objective_function(self, trial, model_type, X, y):
        """Unified objective function for all models"""
        
        if model_type == 'random_forest':
            params = {
                'n_estimators': trial.suggest_int('n_estimators', 100, 500),
                'max_depth': trial.suggest_int('max_depth', 5, 30),
                'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
                'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
                'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2', 0.5, 0.7]),
                'random_state': self.random_state,
                'n_jobs': -1
            }
            model = RandomForestClassifier(**params)
            
        elif model_type == 'xgboost':
            params = {
                'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
                'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
                'max_depth': trial.suggest_int('max_depth', 3, 15),
                'subsample': trial.suggest_float('subsample', 0.6, 1.0),
                'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
                'reg_alpha': trial.suggest_float('reg_alpha', 0, 10),
                'reg_lambda': trial.suggest_float('reg_lambda', 0, 10),
                'random_state': self.random_state,
                'eval_metric': 'logloss',
                'verbosity': 0
            }
            model = xgb.XGBClassifier(**params)
            
        elif model_type == 'lightgbm':
            params = {
                'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
                'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
                'max_depth': trial.suggest_int('max_depth', 3, 15),
                'num_leaves': trial.suggest_int('num_leaves', 10, 300),
                'subsample': trial.suggest_float('subsample', 0.6, 1.0),
                'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
                'reg_alpha': trial.suggest_float('reg_alpha', 0, 10),
                'reg_lambda': trial.suggest_float('reg_lambda', 0, 10),
                'random_state': self.random_state,
                'verbosity': -1,
                'force_col_wise': True
            }
            model = lgb.LGBMClassifier(**params)
            
        elif model_type == 'catboost':
            params = {
                'iterations': trial.suggest_int('iterations', 100, 1000),
                'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
                'depth': trial.suggest_int('depth', 3, 10),
                'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1, 10),
                'random_state': self.random_state,
                'verbose': False
            }
            model = cb.CatBoostClassifier(**params)
            
        elif model_type == 'balanced_rf':
            params = {
                'n_estimators': trial.suggest_int('n_estimators', 100, 500),
                'max_depth': trial.suggest_int('max_depth', 5, 30),
                'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
                'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
                'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2', 0.5, 0.7]),
                'random_state': self.random_state,
                'n_jobs': -1
            }
            model = BalancedRandomForestClassifier(**params)
        
        # Cross-validation
        cv = StratifiedKFold(n_splits=self.cv_folds, shuffle=True, random_state=self.random_state)
        scores = cross_validate(
            model, X, y, 
            cv=cv, 
            scoring=['roc_auc', 'accuracy', 'f1'],
            n_jobs=-1
        )
        
        # Return weighted score (prioritize AUC and F1)
        return 0.5 * scores['test_roc_auc'].mean() + 0.3 * scores['test_f1'].mean() + 0.2 * scores['test_accuracy'].mean()
    
    def optimize_model(self, model_type, X, y):
        """Optimize a specific model type"""
        print(f"Optimizing {model_type}...")
        
        study = optuna.create_study(
            direction='maximize',
            sampler=TPESampler(seed=self.random_state)
        )
        
        study.optimize(
            lambda trial: self.objective_function(trial, model_type, X, y),
            n_trials=self.n_trials,
            show_progress_bar=True
        )
        
        self.best_params[model_type] = study.best_params
        self.study_results[model_type] = study
        
        print(f"Best {model_type} score: {study.best_value:.4f}")
        print(f"Best {model_type} params: {study.best_params}")
        
        return study.best_params, study.best_value
    
    def optimize_all_models(self, X, y, models=['balanced_rf', 'xgboost', 'lightgbm', 'catboost']):
        """Optimize all specified models"""
        results = {}
        
        for model_type in models:
            params, score = self.optimize_model(model_type, X, y)
            results[model_type] = {'params': params, 'score': score}
            
        return results

    # Add this method to your OptunaModelOptimizer class
    def create_optimized_models(self, optimization_results):
        """Create models with optimized parameters"""
        models = {}
        
        for model_type, result in optimization_results.items():
            params = result['params']
            params['random_state'] = self.random_state
            
            if model_type == 'balanced_rf':
                params['n_jobs'] = -1
                models[model_type] = BalancedRandomForestClassifier(**params)
            elif model_type == 'xgboost':
                params['eval_metric'] = 'logloss'
                params['verbosity'] = 0
                models[model_type] = xgb.XGBClassifier(**params)
            elif model_type == 'lightgbm':
                params['verbosity'] = -1
                params['force_col_wise'] = True
                models[model_type] = lgb.LGBMClassifier(**params)
            elif model_type == 'catboost':
                params['verbose'] = False
                models[model_type] = cb.CatBoostClassifier(**params)
                
        return models

# Initialize optimizer
optimizer = OptunaModelOptimizer(n_trials=50, cv_folds=5)  # Reduce trials for demo

# Optimize models (this will take some time)
optimization_results = optimizer.optimize_all_models(
    X_train_selected, Y_train, 
    models=['balanced_rf', 'xgboost', 'lightgbm']  # Start with these three
)

print("\nOptimization completed!")
for model, result in optimization_results.items():
    print(f"{model}: Score = {result['score']:.4f}")

### Step 5: Advanced Model Ensemble and Stacking

In [20]:
class AdvancedEnsemble:
    """
    Advanced ensemble methods including stacking and blending
    """
    
    def __init__(self, base_models, meta_model=None, cv_folds=5, random_state=42):
        self.base_models = base_models
        self.meta_model = meta_model or lgb.LGBMClassifier(random_state=random_state, verbosity=-1)
        self.cv_folds = cv_folds
        self.random_state = random_state
        self.trained_models = {}
        
    def create_optimized_models(self, optimization_results):
        """Create models with optimized parameters"""
        models = {}
        
        for model_type, result in optimization_results.items():
            params = result['params']
            params['random_state'] = self.random_state
            
            if model_type == 'balanced_rf':
                params['n_jobs'] = -1
                models[model_type] = BalancedRandomForestClassifier(**params)
            elif model_type == 'xgboost':
                params['eval_metric'] = 'logloss'
                params['verbosity'] = 0
                models[model_type] = xgb.XGBClassifier(**params)
            elif model_type == 'lightgbm':
                params['verbosity'] = -1
                params['force_col_wise'] = True
                models[model_type] = lgb.LGBMClassifier(**params)
            elif model_type == 'catboost':
                params['verbose'] = False
                models[model_type] = cb.CatBoostClassifier(**params)
                
        return models
    
    def stacking_cv(self, X, y):
        """Generate meta-features using cross-validation"""
        cv = StratifiedKFold(n_splits=self.cv_folds, shuffle=True, random_state=self.random_state)
        meta_features = np.zeros((X.shape[0], len(self.base_models)))
        
        for fold, (train_idx, val_idx) in enumerate(cv.split(X, y)):
            X_train_fold = X.iloc[train_idx]
            y_train_fold = y.iloc[train_idx]
            X_val_fold = X.iloc[val_idx]
            
            for i, (name, model) in enumerate(self.base_models.items()):
                # Clone and train model
                model_clone = clone(model)
                model_clone.fit(X_train_fold, y_train_fold)
                
                # Predict on validation set
                pred_proba = model_clone.predict_proba(X_val_fold)[:, 1]
                meta_features[val_idx, i] = pred_proba
                
        return meta_features
    
    def fit_stacking(self, X, y):
        """Fit stacking ensemble"""
        print("Generating meta-features...")
        meta_features = self.stacking_cv(X, y)
        
        print("Training meta-model...")
        self.meta_model.fit(meta_features, y)
        
        # Train base models on full data
        print("Training base models on full data...")
        for name, model in self.base_models.items():
            model.fit(X, y)
            self.trained_models[name] = model
            
        return self
    
    def predict_stacking(self, X):
        """Predict using stacking ensemble"""
        meta_features = np.zeros((X.shape[0], len(self.base_models)))
        
        for i, (name, model) in enumerate(self.trained_models.items()):
            pred_proba = model.predict_proba(X)[:, 1]
            meta_features[:, i] = pred_proba
            
        return self.meta_model.predict(meta_features)
    
    def predict_proba_stacking(self, X):
        """Predict probabilities using stacking ensemble"""
        meta_features = np.zeros((X.shape[0], len(self.base_models)))
        
        for i, (name, model) in enumerate(self.trained_models.items()):
            pred_proba = model.predict_proba(X)[:, 1]
            meta_features[:, i] = pred_proba
            
        return self.meta_model.predict_proba(meta_features)
    
    def weighted_average_ensemble(self, X, weights=None):
        """Simple weighted average ensemble"""
        if weights is None:
            weights = np.ones(len(self.trained_models)) / len(self.trained_models)
            
        predictions = np.zeros(X.shape[0])
        
        for i, (name, model) in enumerate(self.trained_models.items()):
            pred_proba = model.predict_proba(X)[:, 1]
            predictions += weights[i] * pred_proba
            
        return (predictions > 0.5).astype(int), predictions

# Create optimized models
optimized_models = optimizer.create_optimized_models(optimization_results)

# Create ensemble
ensemble = AdvancedEnsemble(optimized_models)

# Fit stacking ensemble
ensemble.fit_stacking(X_train_selected, Y_train)

print("Ensemble training completed!")

Generating meta-features...


  File "e:\miniconda3\Lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
        "wmic CPU Get NumberOfCores /Format:csv".split(),
        capture_output=True,
        text=True,
    )
  File "e:\miniconda3\Lib\subprocess.py", line 556, in run
    with Popen(*popenargs, **kwargs) as process:
         ~~~~~^^^^^^^^^^^^^^^^^^^^^^
  File "e:\miniconda3\Lib\subprocess.py", line 1038, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
    ~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
                        pass_fds, cwd, env,
                        ^^^^^^^^^^^^^^^^^^^
    ...<5 lines>...
                        gid, gids, uid, umask,
                        ^^^^^^^^^^^^^^^^^^^^^^
                        start_new_session, process_group)
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "e:\miniconda3\Lib\subprocess.py", line 1550, in _execute_child
    hp, ht

Training meta-model...
Training base models on full data...
Ensemble training completed!


### Step 6: Comprehensive Model Evaluation

In [28]:
class ComprehensiveEvaluator:
    """
    Comprehensive model evaluation with multiple metrics and visualizations
    """
    
    def __init__(self):
        self.results = {}
        
    def calculate_metrics(self, y_true, y_pred, y_pred_proba=None):
        """Calculate comprehensive metrics"""
        from sklearn.metrics import (
            accuracy_score, precision_score, recall_score, f1_score,
            roc_auc_score, matthews_corrcoef, confusion_matrix,
            classification_report
        )
        
        metrics = {
            'accuracy': accuracy_score(y_true, y_pred),
            'precision': precision_score(y_true, y_pred),
            'recall': recall_score(y_true, y_pred),
            'f1': f1_score(y_true, y_pred),
            'mcc': matthews_corrcoef(y_true, y_pred)
        }
        
        if y_pred_proba is not None:
            metrics['auc'] = roc_auc_score(y_true, y_pred_proba)
            
        # Confusion matrix components
        tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
        metrics.update({
            'sensitivity': tp / (tp + fn),
            'specificity': tn / (tn + fp),
            'tp': tp, 'tn': tn, 'fp': fp, 'fn': fn
        })
        
        return metrics
    
    def cross_validation_evaluation(self, model, X, y, cv_folds=5):
        """Comprehensive cross-validation evaluation"""
        cv = StratifiedKFold(n_splits=cv_folds, shuffle=True, random_state=42)
        
        cv_results = {
            'accuracy': [], 'precision': [], 'recall': [], 'f1': [],
            'auc': [], 'mcc': [], 'sensitivity': [], 'specificity': []
        }
        
        for train_idx, val_idx in cv.split(X, y):
            X_train_fold = X.iloc[train_idx]
            y_train_fold = y.iloc[train_idx]
            X_val_fold = X.iloc[val_idx]
            y_val_fold = y.iloc[val_idx]
            
            # Train and predict
            model_clone = clone(model)
            model_clone.fit(X_train_fold, y_train_fold)
            y_pred = model_clone.predict(X_val_fold)
            y_pred_proba = model_clone.predict_proba(X_val_fold)[:, 1]
            
            # Calculate metrics
            fold_metrics = self.calculate_metrics(y_val_fold, y_pred, y_pred_proba)
            
            for metric in cv_results:
                if metric in fold_metrics:
                    cv_results[metric].append(fold_metrics[metric])
        
        # Calculate means and stds
        cv_summary = {}
        for metric, values in cv_results.items():
            cv_summary[f'{metric}_mean'] = np.mean(values)
            cv_summary[f'{metric}_std'] = np.std(values)
            
        return cv_summary
    
    def evaluate_model(self, model, X_train, y_train, X_test, y_test, model_name):
        """Complete model evaluation"""
        print(f"Evaluating {model_name}...")
        
        # Cross-validation results
        cv_results = self.cross_validation_evaluation(model, X_train, y_train)
        
        # Train on full training set and test
        model.fit(X_train, y_train)
        y_pred_test = model.predict(X_test)
        y_pred_proba_test = model.predict_proba(X_test)[:, 1]
        
        # Test set metrics
        test_metrics = self.calculate_metrics(y_test, y_pred_test, y_pred_proba_test)
        
        # Store results
        self.results[model_name] = {
            'cv_results': cv_results,
            'test_metrics': test_metrics,
            'predictions': {
                'y_pred': y_pred_test,
                'y_pred_proba': y_pred_proba_test
            }
        }
        
        return cv_results, test_metrics
    
    def create_results_dataframe(self):
        """Create comprehensive results DataFrame"""
        data = []
        
        for model_name, result in self.results.items():
            row = {'Model': model_name}
            
            # Add CV results (only if they exist)
            if 'cv_results' in result:
                for metric, value in result['cv_results'].items():
                    row[f'CV_{metric}'] = value
            else:
                # Fill with NaN for missing CV results
                cv_metrics = ['accuracy_mean', 'accuracy_std', 'precision_mean', 'precision_std', 
                            'recall_mean', 'recall_std', 'f1_mean', 'f1_std', 'auc_mean', 'auc_std',
                            'mcc_mean', 'mcc_std', 'sensitivity_mean', 'sensitivity_std', 
                            'specificity_mean', 'specificity_std']
                for metric in cv_metrics:
                    row[f'CV_{metric}'] = np.nan
                
            # Add test results
            for metric, value in result['test_metrics'].items():
                if metric not in ['tp', 'tn', 'fp', 'fn']:
                    row[f'Test_{metric}'] = value
                    
            data.append(row)
            
        return pd.DataFrame(data)
    
    def plot_roc_curves(self, X_test, y_test):
        """Plot ROC curves for all models"""
        fig = go.Figure()
        
        for model_name, results in self.results.items():
            y_pred_proba = results['predictions']['y_pred_proba']
            
            from sklearn.metrics import roc_curve
            fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
            auc_score = results['test_metrics']['auc']
            
            fig.add_trace(go.Scatter(
                x=fpr, y=tpr,
                mode='lines',
                name=f'{model_name} (AUC = {auc_score:.3f})',
                line=dict(width=3)
            ))
        
        # Add diagonal line
        fig.add_trace(go.Scatter(
            x=[0, 1], y=[0, 1],
            mode='lines',
            name='Random (AUC = 0.5)',
            line=dict(dash='dash', color='gray')
        ))
        
        fig.update_layout(
            title='ROC Curves Comparison',
            xaxis_title='False Positive Rate',
            yaxis_title='True Positive Rate',
            width=800, height=600
        )
        
        fig.show()
        
    def plot_feature_importance(self, model, feature_names, model_name, top_n=20):
        """Plot feature importance"""
        if hasattr(model, 'feature_importances_'):
            importances = model.feature_importances_
        elif hasattr(model, 'coef_'):
            importances = np.abs(model.coef_[0])
        else:
            print(f"Cannot extract feature importance for {model_name}")
            return
            
        feature_imp = pd.DataFrame({
            'feature': feature_names,
            'importance': importances
        }).sort_values('importance', ascending=False)
        
        top_features = feature_imp.head(top_n)
        
        fig = px.bar(
            top_features.iloc[::-1],  # Reverse for better visualization
            x='importance',
            y='feature',
            orientation='h',
            title=f'Top {top_n} Feature Importances - {model_name}'
        )
        
        fig.update_layout(height=600)
        fig.show()

# Evaluate individual models
evaluator = ComprehensiveEvaluator()

# Evaluate optimized models
for model_name, model in optimized_models.items():
    evaluator.evaluate_model(
        model, X_train_selected, Y_train, X_test_selected, Y_test, model_name
    )

# Evaluate ensemble
ensemble_pred = ensemble.predict_stacking(X_test_selected)
ensemble_pred_proba = ensemble.predict_proba_stacking(X_test_selected)[:, 1]

# Add ensemble results manually
ensemble_test_metrics = evaluator.calculate_metrics(Y_test, ensemble_pred, ensemble_pred_proba)

# # cross-validate ensemble
# cv_results = evaluator.cross_validation_evaluation(ensemble, X_train_selected, Y_train)

evaluator.results['Stacking_Ensemble'] = {
    # 'cv_results': cv_results,
    'test_metrics': ensemble_test_metrics,
    'predictions': {
        'y_pred': ensemble_pred,
        'y_pred_proba': ensemble_pred_proba
    }
}

# Create results summary
results_df = evaluator.create_results_dataframe()
print("\nModel Comparison Results:")
print(results_df.round(4))

# Plot ROC curves
evaluator.plot_roc_curves(X_test_selected, Y_test)

Evaluating balanced_rf...
Evaluating xgboost...
Evaluating lightgbm...

Model Comparison Results:
               Model  CV_accuracy_mean  CV_accuracy_std  CV_precision_mean  \
0        balanced_rf            0.8240           0.0423             0.6624   
1            xgboost            0.8281           0.0136             0.7660   
2           lightgbm            0.8239           0.0214             0.7190   
3  Stacking_Ensemble               NaN              NaN                NaN   

   CV_precision_std  CV_recall_mean  CV_recall_std  CV_f1_mean  CV_f1_std  \
0            0.0993          0.6362         0.0832      0.6434     0.0680   
1            0.0362          0.4413         0.0667      0.5567     0.0552   
2            0.0559          0.4750         0.0519      0.5712     0.0511   
3               NaN             NaN            NaN         NaN        NaN   

   CV_auc_mean  ...  CV_specificity_mean  CV_specificity_std  Test_accuracy  \
0       0.8507  ...               0.8857      